In [ ]:
# ================================================================
# Advanced Time Series Forecasting with Neural Networks
# Probabilistic Forecasting using Quantile Regression LSTM
# ================================================================

# ========================
# 1. IMPORT LIBRARIES
# ========================
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt


# ========================
# 2. GENERATE MULTIVARIATE TIME SERIES (5 correlated features)
# ========================
np.random.seed(42)
T = 1000
t = np.arange(T)

season = 2*np.sin(2*np.pi*t/50)  # seasonal
regime = np.where(t < 600, 0.02*t, 0.02*600 + 0.05*(t-600))  # nonlinear regime shift

# 5×5 covariance matrix
cov = np.array([
    [1.0, 0.7, 0.5, 0.4, 0.3],
    [0.7, 1.0, 0.6, 0.5, 0.2],
    [0.5, 0.6, 1.0, 0.4, 0.4],
    [0.4, 0.5, 0.4, 1.0, 0.6],
    [0.3, 0.2, 0.4, 0.6, 1.0]
])
noise = np.random.multivariate_normal(np.zeros(5), cov, size=T)

X = np.zeros((T, 5))
for i in range(5):
    X[:,i] = 0.5*season + 0.3*regime + noise[:,i]

df = pd.DataFrame(X, columns=[f"feat_{i+1}" for i in range(5)])
df["target"] = df["feat_1"].shift(-1)
df = df.dropna().reset_index(drop=True)

print("Dataset shape:", df.shape)


# ========================
# 3. TRAIN/TEST SPLIT + SCALING
# ========================
scaler = StandardScaler()
scaled = scaler.fit_transform(df.values)

train_size = 800
train_data = scaled[:train_size]
test_data = scaled[train_size:]

WINDOW = 20


# ========================
# 4. PYTORCH DATASET
# ========================
class SeqDataset(Dataset):
    def __init__(self, data, window):
        X, y = [], []
        for i in range(len(data)-window):
            X.append(data[i:i+window,:5])
            y.append(data[i+window,5])
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

train_ds = SeqDataset(train_data, WINDOW)
test_ds = SeqDataset(test_data, WINDOW)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)


# ========================
# 5. PROBABILISTIC LSTM (Quantile Regression)
# ========================
class QuantileLSTM(nn.Module):
    def __init__(self, n_features, hidden=64, n_quantiles=3):
        super().__init__()
        self.lstm = nn.LSTM(n_features, hidden, batch_first=True)
        self.fc = nn.Linear(hidden, n_quantiles)

    def forward(self, x):
        out,_ = self.lstm(x)
        return self.fc(out[:,-1,:])

def pinball(y_true, y_pred, q):
    err = y_true - y_pred
    return torch.max((q-1)*err, q*err).mean()

quantiles = torch.tensor([0.1, 0.5, 0.9])
model_q = QuantileLSTM(5)
opt_q = torch.optim.Adam(model_q.parameters(), lr=1e-3)


# ========================
# 6. TRAIN QUANTILE MODEL
# ========================
print("\nTraining Quantile_LSTM...")
for epoch in range(30):
    model_q.train()
    loss_total = 0
    for Xb, yb in train_loader:
        opt_q.zero_grad()
        preds = model_q(Xb)
        loss = sum(pinball(yb, preds[:,i], quantiles[i]) for i in range(3))
        loss.backward()
        opt_q.step()
        loss_total += loss.item()
    print(f"Epoch {epoch+1:02d}, Loss={loss_total:.4f}")


# ========================
# 7. BASELINE MODEL (Point LSTM)
# ========================
class PointLSTM(nn.Module):
    def __init__(self, n_features, hidden=64):
        super().__init__()
        self.lstm = nn.LSTM(n_features, hidden, batch_first=True)
        self.fc = nn.Linear(hidden, 1)

    def forward(self, x):
        out,_ = self.lstm(x)
        return self.fc(out[:,-1,:]).squeeze()

model_base = PointLSTM(5)
opt_b = torch.optim.Adam(model_base.parameters(), lr=1e-3)
mse_loss = nn.MSELoss()

print("\nTraining Baseline LSTM...")
for epoch in range(30):
    model_base.train()
    epoch_loss = 0
    for Xb, yb in train_loader:
        opt_b.zero_grad()
        pred = model_base(Xb)
        loss = mse_loss(pred, yb)
        loss.backward()
        opt_b.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1:02d}, Loss={epoch_loss:.4f}")


# ========================
# 8. EVALUATION FUNCTIONS
# ========================
def evaluate_quantile(model, loader):
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for Xb, yb in loader:
            p = model(Xb)
            preds.append(p.numpy())
            trues.append(yb.numpy())
    return np.concatenate(preds), np.concatenate(trues)

def evaluate_point(model, loader):
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for Xb, yb in loader:
            p = model(Xb)
            preds.append(p.numpy())
            trues.append(yb.numpy())
    return np.concatenate(preds), np.concatenate(trues)

def coverage(preds, trues):
    L = preds[:,0]
    H = preds[:,2]
    return np.mean((trues >= L) & (trues <= H))

def interval_width(preds):
    return np.mean(preds[:,2] - preds[:,0])


# ========================
# 9. RUN EVALUATION
# ========================
preds_q, trues = evaluate_quantile(model_q, test_loader)
preds_b, trues_b = evaluate_point(model_base, test_loader)

rmse_q = mean_squared_error(trues, preds_q[:,1], squared=False)
mae_q = mean_absolute_error(trues, preds_q[:,1])

rmse_b = mean_squared_error(trues_b, preds_b, squared=False)
mae_b = mean_absolute_error(trues_b, preds_b)

cov80 = coverage(preds_q, trues)
iw = interval_width(preds_q)

print("\n==================== RESULTS ====================")
print(f"Baseline LSTM: RMSE={rmse_b:.4f}, MAE={mae_b:.4f}")
print(f"Quantile LSTM: RMSE={rmse_q:.4f}, MAE={mae_q:.4f}")
print(f"Coverage (80% interval) = {cov80:.4f}")
print(f"Average interval width = {iw:.4f}")
print("=================================================")


# ========================
# 10. PLOTS
# ========================
plt.figure(figsize=(12,6))
plt.plot(trues[:200], label="True", color="black")
plt.plot(preds_q[:200,1], label="Median Forecast", color="blue")
plt.fill_between(np.arange(200), preds_q[:200,0], preds_q[:200,2],
                 color="lightblue", alpha=0.5, label="80% interval")
plt.title("Probabilistic Forecast (First 200 steps)")
plt.legend()
plt.show()
